In [3]:
pip install selenium

     ------------------------------------ 986.0/986.0 kB 538.2 kB/s eta 0:00:00
     -------------------------------------- 359.0/359.0 kB 1.3 MB/s eta 0:00:00
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
     -------------------------------------- 58.2/58.2 kB 772.8 kB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import selenium

In [17]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
url="https://www.linkedin.com/?trk=guest_homepage-basic_nav-header-logo"

In [5]:
wd = webdriver.Chrome('C:/Users/ANOOP/Documents/Chromedriver/chromedriver')
wd.get(url)

C:\Users\ANOOP\AppData\Local\Temp\ipykernel_10984\2844372254.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('C:/Users/ANOOP/Documents/Chromedriver/chromedriver')


In [28]:
# job_lists = wd.find_element(By.CLASS_NAME,"jobs-search__results-list")
# jobs = job_lists.find_elements(By.TAG_NAME,"li")

In [6]:
username=wd.find_element(By.ID,"session_key")
username.send_keys("ranjananup76@gmail.com")

In [7]:
password=wd.find_element(By.ID,"session_password")
password.send_keys("hello@098")

In [8]:
login_button=wd.find_element(By.CLASS_NAME,"sign-in-form__submit-button")
login_button.click()

In [ ]:
# https://www.linkedin.com/jobs/search/?currentJobId=3270791709&distance=25&geoId=105214831&keywords=data%20analyst

In [ ]:
# https://www.linkedin.com/jobs/search/?currentJobId=3264450260&distance=25&geoId=105214831&keywords=data%20analyst&start=25

In [ ]:
# https://www.linkedin.com/jobs/search/?currentJobId=3270791709&distance=25&f_WT=1&geoId=105214831&keywords=data%20analyst

In [9]:
jobtype = '1' #On-site
keyword = 'data%20analyst' # "%20" = space 
page = '0' #start at 0. Goes in multiples of 25 
#URL
URL1 = 'https://www.linkedin.com/jobs/search/?f_WT='+jobtype+'&geoId=105214831&keywords='+keyword+'&start='+page

In [ ]:
# 'https://www.linkedin.com/jobs/search/?f_WT=1&geoId=105214831&keywords=data%20analyst&start=0'

In [26]:
def getLinkedinJobs(wd,keyword,jobType,maxPageNumSearch):
    actions = ActionChains(wd)
    jobtype = str(jobType) #1 = Onsite
    keyword = keyword.replace(' ','%20') # %20 = space
    jobData = [['title','company','companyLink','location','jobLink','posterName','posterProfileLink','jobDescription']]
    for p in range(maxPageNumSearch):
        page = str(p*25)
        #print(page)
        wd.get('https://www.linkedin.com/jobs/search/?f_WT='+jobtype+'&geoId=105214831&keywords='+keyword+'&start='+page)
        time.sleep(3)
        #Search for all job postings shown
        jobs = wd.find_elements(By.CLASS_NAME,'jobs-search-results__list-item')
        i=0
        for j in jobs:
            #print(i)
            i+=1
            time.sleep(2)
            actions.move_to_element(j).perform()
            j.find_element(By.TAG_NAME,'img').click() #Click on the image so it doesnt misclick a URL
            jobData.append(getJobInfo(wd))
    df = pd.DataFrame(jobData[1:],columns=jobData[0])
    #Drop any duplicate
    df.drop_duplicates(subset = ['jobLink'],inplace=True)
    return df

In [19]:
def getJobInfo(wd):
    title = ''
    company = ''
    companyLink = ''
    location = ''
    jobLink = ''
    posterName = ''
    posterProfileLink = ''
    jobDescription = ''
    try:
        title = wd.find_element(By.CLASS_NAME,'jobs-unified-top-card__content--two-pane').text.split('\n')[0]
    except:
        pass
    try:
        company = wd.find_element(By.CLASS_NAME,'jobs-unified-top-card__company-name').text
    except:
        pass
    try:
        companyLink = wd.find_element(By.CLASS_NAME,'jobs-unified-top-card__company-name').find_element(By.TAG_NAME,'a').get_attribute('href')
    except:
        pass
    try:
        location = wd.find_element(By.CLASS_NAME,'jobs-unified-top-card__bullet').text
    except:
        pass
    try:
        jobLink = wd.find_element(By.CLASS_NAME,'jobs-unified-top-card__content--two-pane').find_element(By.TAG_NAME,'a').get_attribute('href')
    except:
        pass
    try:
        posterName = wd.find_element(By.CLASS_NAME,'jobs-poster__name').text
    except:
        pass
    try:
        posterProfileLink = wd.find_element(By.CLASS_NAME,'jobs-poster__name-link').get_attribute('href')
    except:
        pass
    try:
        jobDescription = wd.find_element(By.CLASS_NAME,'jobs-box__html-content').find_element(By.TAG_NAME,'span').text.replace('\n',' ')
    except:
        pass
    return [title,company,companyLink,location,jobLink,posterName,posterProfileLink,jobDescription]

In [29]:
a=getLinkedinJobs(wd,'data analyst','1',10)

In [32]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              250 non-null    object
 1   company            250 non-null    object
 2   companyLink        250 non-null    object
 3   location           250 non-null    object
 4   jobLink            250 non-null    object
 5   posterName         250 non-null    object
 6   posterProfileLink  250 non-null    object
 7   jobDescription     250 non-null    object
dtypes: object(8)
memory usage: 17.6+ KB


In [36]:
a.drop(['posterName','posterProfileLink'],axis=1,inplace=True)

In [41]:
a["company"].unique()

array(['Giant Eagle GCC', 'ABB', 'Target', 'Honeywell',
       'Bosch Global Software Technologies', 'Happay', 'Cargill', 'Wipro',
       'LKQ Corporation', 'Autodesk', 'Rystad Energy',
       'Accenture in India', 'Open Financial Technologies',
       'Fidelity Investments', 'BD', 'Optum', 'Gensler', 'Nielsen',
       'twid', 'SuperK', 'Bread Financial', 'Vedantu', 'TMF Group',
       'AKS ProTalent', 'Simplify360', 'FloBiz', 'Indium Software',
       'DocsApp', 'Dunzo', 'Qualcomm', 'Eloelo', 'Pankhuri', 'Bayer',
       'ISG (Information Services Group)', 'Hewlett Packard Enterprise',
       'Netradyne', 'JLL', 'Yara International', 'SKILLETY', 'Chubb',
       'Inference Labs', 'Cephas Consultancy Services Private Limited',
       'HuntingCube Recruitment Solutions', 'Next Labs',
       'Ezetap by Razorpay', 'Logitech', 'Rolls-Royce',
       'Millennium IMS Private Limited', 'NTT DATA Services',
       'SymphonyAI Retail CPG', 'grofers', 'Becton Dickenson',
       "Moody's Corporation